In [1]:
# data manager and analysis
import vodex as vx
import numan as nu
import pandas as pd

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-ssg79bxz because the default path (/home/ply/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Project structure: 

Provide the project folder with the "processed" folder created in the previous notebook. 

As you keep going with the analysis, the folder will have the following structure: 

```
...........................................................
....................... DONE in 01 ........................
...........................................................
processed                                               ...
│   experiment.json <-----------------------------------... the file that contains everything about the experiment, you are creating it once and will be reusing ever after
│   experiment_dff.json <-------------------------------... everything about the experiment, but loads from the dff movie, not from the raw data                   ...
└───dff_movie  <----------------------------------------...the dff movie :)
│   │   dff_movie_0000.tif                              ...
│   │   dff_movie_0001.tif                              ...
│   │   ...                                             ...
│..........................................................
│...................... DONE in 02 ........................
│..........................................................
│                                                       ...
└───tscore_volumes  <-----------------------------------... t-score tif files
│   │   tscore_SvB.tif <--------------------------------... t-score Stimuli vs Blank
│   │                                                   ...
│..........................................................
│...................... DONE : MANUAL .....................
│..........................................................
└───spots                                               ...
│   └───imaris  <---------------------------- ATTENTION ... You need to put stuff generated by imaris into this folder!!!                                         ...
│       │   └───tscore_SvB_Statistics                   ...
│       │       │     tscore_SvB_Position.csv           ...
│       │       │     tscore_SvB_Diameter.csv           ...
│       │       │     ...                               ...
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++ WILL BE DONE in this notebook +++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│   │                                                   +++
│   └───signals  <--------------------------------------+++ json files with the extracted signals, also will have the group info after you added it                 +++
│       │   spots_SvB.json                              +++
│       │                                               +++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│   │
│   └───reports  <---------------------------------- tiffs and pdf with the cells significant in any pairwise comparison
│       └───all_significant  <---------------------- tiffs and pdf with all significant in any way cells
│           │   └───signals  <---------------------- pdfs with signals
│           │       │     ...
│           │   └───images <------------------------ tif masks
│           │       │     ...
│       └───groupped  <----------------------------- tiffs and pdf where the cells are groupped based on signal shape .. or anything else you want
│           │   readme.txt  <----------------------- ATTENTION : you need to describe the groups
│           │   └───signals  <---------------------- pdfs with signals
│           │       │     ...
│           │   └───images  <----------------------- tif masks
│           │       │     ...
```

Also, if the processed folder should already exist if you created dff movies. If the folder doesn't exist, it will complain ...

# Make sure you have the imaris files in the right folders... 

You can use any way to segemnt the images, we need position and diameter


```
processed
│!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
│!!!!!!!!!!!!!!!! ATTENTION: MANUAL STEP !!!!!!!!!!!!!!!!!!
│!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
└───spots                                               !!!
│   └───imaris  <---------------------------- ATTENTION !!! You need to put stuff generated by imaris into this folder!!!                                         !!!
│       │   └───tscore_SvB_max_Statistics               !!!
│       │       │     tscore_SvB_max_Position.csv       !!!
│       │       │     tscore_SvB_max_Diameter.csv       !!!
│       │       │     ...                               !!!
│!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
│!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
│!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
```

# Set project folder

In [2]:
project_folder = "/home/ply/repos/numan/notebooks/vodex_si/data/"
project = nu.Project(project_folder)

project.check_exists("processed")
project.check_exists("processed/spots/imaris")

project.activate("processed")

# Load experiment with the raw data:

In [6]:
experiment = vx.Experiment.load("experiment_raw.db")

# add the annotations to the experiment
numerosity_df = pd.read_csv("/home/ply/repos/numan/notebooks/vodex_si/data/annotations/numerosity_annotation.csv", index_col=0)
experiment.add_annotations_from_df(numerosity_df, cycles=True) 

# save the experiment
experiment.save("experiment_raw.db")

In [7]:
numerosity_df

,duration_frames,name,group,description
duration_volumes,,,,
3,201,b,number,"blank, no dots"
1,67,d4,number,4 dots on the screen
9,603,b,number,"blank, no dots"
1,67,d3,number,3 dots on the screen
7,469,b,number,"blank, no dots"
1,67,d5,number,5 dots on the screen
5,335,b,number,"blank, no dots"
1,67,d2,number,2 dots on the screen
7,469,b,number,"blank, no dots"


## Extract spot signals
This is done for each set of segmentations.

Make the directory for signals:

In [8]:
project.create("processed/spots/signals")


Get a wrapper function

In [9]:
# if you run out of memory, reduce the batch size (this is how many timepoints are being 
BATCH_SIZE = 100
RESOLUTION = [2.4, 1.17, 1.17]

def load_and_save_signals(points_file, diameter_file, group_tag):
    # Load spots from the Position and Diameter
    spots = nu.Spots.from_imaris(points_file,diameter_file,resolution = RESOLUTION, units='phs')
    # extract signal ( this takes a long time)
    spots.get_signals(volumes="all", experiment=experiment, batch_size=BATCH_SIZE, traces_type="raw")
    spots.to_json(f"spots/signals/spots_{group_tag}.json")

Extract the signals and save, sorry it outputs a lot of stuff ... 

In [12]:
tag = "SvB_max"
print(f'{tag}___________________________________________________________________________________________')
points_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Position.csv"
diameter_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Diameter.csv"
load_and_save_signals(points_file, diameter_file, tag)

SvB_max___________________________________________________________________________________________


/home/ply/miniconda3/envs/numan/lib/python3.11/site-packages/vodex/experiment.py:601: UserWarning: list_volumes will be removed in vodex 1.1.0 use volumes property instead.
  warnings.warn(f"list_volumes will be removed in vodex 1.1.0 use volumes property instead.")
100%|██████████| 6700/6700 [00:01<00:00, 4414.13frames/s]


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
experiment.close()